# Demonstrating LUSID Support for Mortgage Backed Securities

In this notebook we show how to:
 - book an MBS bond into LUSID 
 - query past and future cashflows and verify cashflow entitlements
 - query accrued interest and current notional
 - value a portfolio that has this bond

The example is based off a real Mortgage Backed Security bond issued by Fannie Mae (Isin: US3136AKAD59).

## Table of Contents:
- 1. [Set up the environment required for valuation](#1.-Set-Up-Environment)
- 2. [Create portfolios for the tests](#2.-Create-Portfolios)
- 3. [Creating an MBS instrument](#3.-Creating-an-MBS-Instrument)
- 4. [Pool Factors](#4.-Pool-Factors)
- 5. [Transactions](#5.-Transactions)
- 6. [Cash flows and Entitlements](#6.-Cash-Flows-and-Entitlements)
- 7. [Valuation & PnL](#7.-Valuations)


# 1. Set Up Environment

First we set up the local variables and the test environment:

In [ ]:
# Import generic non-LUSID packages
import os
import numpy as np
import datetime as dt
import pandas as pd
import json
import pytz
import itertools
import concurrent.futures
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm
from lusid.utilities import ApiClientFactory
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import format_transactions_response

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidjam import RefreshingToken

# Set pandas to display a nice number formatting
pd.options.display.float_format = '{:,.2f}'.format

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)

In [ ]:
# LUSID Variable Definitions
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lu.api.InstrumentsApi)
portfolios_api = api_factory.build(lu.api.PortfoliosApi)
recipes_api = api_factory.build(lu.api.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lu.api.AggregationApi)
quotes_api = api_factory.build(lu.api.QuotesApi)
calendar_api = api_factory.build(lu.api.CalendarsApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
scope = "TESTMBS"

# 2. Create Portfolios

We need to book transactions into portfolios. So the first step is to create them!

To ensure we start from a clean slate, let's begin by deleting any portfolios from previous runs, if they exist.

In [ ]:
def delete_portfolio(portfolio):

    try:
        portfolios_api.delete_portfolio(
            scope=scope,
            code=portfolio
        )
        print(f'Portfolio "{scope}/{portfolio}" deleted')
        return 1

    except lu.ApiException as e:
        detail = json.loads(e.body)
        if detail["code"] == 109:
            print(f'Portfolio "{scope}/{portfolio}" did not exist')
            return 0
        else:
            raise e

In [ ]:
delete_portfolio("portfolio_1")
delete_portfolio("portfolio_2")

And now we create the two new portfolios required for our demo.

In [ ]:
def create_portfolio(portfolio):

    try:
        transaction_portfolios_api.create_portfolio(
            scope=scope,
            create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
                display_name=portfolio,
                code=portfolio,
                base_currency="USD",
                created=datetime(year=1900, month=1, day=1, tzinfo=pytz.UTC),
                instrument_scopes=[f"{scope}"],
            ),
        )
        print(f'Portfolio "{scope}/{portfolio}" created')
        return 1

    except lu.ApiException as e:
        detail = json.loads(e.body)
        if detail["code"] == 112:
            print(f'Portfolio "{scope}/{portfolio}" already exists')
            return 0
        else:
            print(e)
            raise e

In [ ]:
create_portfolio("portfolio_1")
create_portfolio("portfolio_2")

# 3. Creating an MBS Instrument

Now let's define a function that takes in the information required to build a Fannie Mae Fixed Rate MBS and upserts it to lusid.

In [ ]:
def create_fixed_rate_Fannie_Mae_MBS(
   bond_name,
   ISIN,
   currency,
   start_date,
   maturity_date,
   original_notional,
   coupon,
   pay_delay,
   payment_frequency,
   roll_convention,
   stub_type
):
    bond_flow_conventions = lm.FlowConventions(
            currency = currency,
            payment_frequency = payment_frequency,
            roll_convention = roll_convention,
            day_count_convention = 'Thirty360',
            payment_calendars = [],
            reset_calendars = []
    )

    pay_delay_information = lm.ExDividendConfiguration(
        ex_dividend_days = pay_delay,
        apply_thirty360_pay_delay = True
    )

    fixed_schedule = lm.FixedSchedule(
        schedule_type = "FixedSchedule",
        start_date = start_date,
        maturity_date = maturity_date,
        coupon_rate = coupon,
        notional = original_notional,
        payment_currency = currency,
        stub_type = 'None',
        flow_conventions = bond_flow_conventions,
        ex_dividend_configuration = pay_delay_information
    )

    bond = lm.ComplexBond(
        asset_backed = True,
        asset_pool_identifier = ISIN,
        instrument_type = "ComplexBond",
        schedules = [fixed_schedule]
    )

    # define the instrument to be upserted
    bond_definition = lm.InstrumentDefinition(
        name=bond_name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(ISIN)},
        definition=bond,
    )

    # upsert the instrument
    upsert_request = {ISIN: bond_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    bond_luid = upsert_response.values[ISIN].lusid_instrument_id
    print(f"Bond {ISIN} upserted successfully - LUID is {bond_luid}")

We defined using the lusid SDK a function that creates a typical case of an MBS from relevant trade information. This function can be very handy if we need to create several MBSs, as we encapsulate the handling of all the required complex bond objects into a single function call.

Now, before we upsert any instrument, to ensure we start our demo from a clean slate (similar to what we did to portfolios), let's delete all existing instruments in LUSID in our scope.

In [ ]:
resp = instruments_api.list_instruments(
    scope = scope,
    limit = 2000
)

df_LUIDs = [itm.identifiers['LusidInstrumentId'] for itm in resp.values]

if len(df_LUIDs) > 0:
    resp = instruments_api.delete_instruments(
        request_body = df_LUIDs,
        scope = scope
    )

And here we create the MBS of our interest (US3136AKAD59):

In [ ]:
coupon_rate = 0.025
bond_name = 'FED NATIONAL MTGE ASSOC 2014-33'
ISIN = 'US3136AKAD59'
currency = 'USD'
start_date = datetime(2014, 5, 1, 00, tzinfo=pytz.utc)
maturity_date = datetime(2029, 9, 25, 00, tzinfo=pytz.utc)
original_notional = 1000000
pay_delay = 24
payment_frequency = '1M'
roll_convention = 'Day25'
stub_type = 'None'

create_fixed_rate_Fannie_Mae_MBS(
   bond_name,
   ISIN,
   currency,
   start_date,
   maturity_date,
   original_notional,
   coupon_rate,
   pay_delay,
   payment_frequency,
   roll_convention,
   stub_type
)

# 4. Pool Factors

Additionally, we also need to upsert the required pool factors. 

In [ ]:
pool_factor_data = [["US3136AKAD59", "2023-05-01", 0.100641],
                    ["US3136AKAD59", "2023-06-01", 0.098082],
                    ["US3136AKAD59", "2023-07-01", 0.095056]]
pool_factors = pd.DataFrame(pool_factor_data, columns=["ISIN","Date","Factor"])

In [ ]:
def upsert_factor(ISIN, date, factor):
    quotes = {
            0: lm.UpsertQuoteRequest(
            quote_id=lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=ISIN,
                    instrument_id_type="Isin",
                    quote_type="PoolFactor",
                    field="mid",
                ),
                effective_at=date.isoformat()+'Z',
            ),
            metric_value=lm.MetricValue(value=factor, unit="ratio"),
            scale_factor=1
        )
    }
    upsert_response = quotes_api.upsert_quotes(scope=scope, request_body=quotes)
    if upsert_response.failed:
        print(f"Failed to upsert {factor} for {ISIN} at {date}.")
    else:
        print(f"Upserting {factor} for {ISIN} at {date} completed successfully.")


In [ ]:
for idx in pool_factors.index:
    upsert_factor(
        pool_factors['ISIN'][idx], 
        datetime.strptime(pool_factors['Date'][idx],"%Y-%m-%d"), 
        pool_factors['Factor'][idx])


# 5. Transactions

In this demo, first we will:
 - Buy 8,000,000 USD (original notional value) of the bond on the 28-May-2023 and add this position to portfolio 1
 - Buy 8,000,000 USD (original notional value) unit of the same bond on the 03-Jun-2023 and add this position to portfolio 2
 - Compare the different cashflow profiles between portfolio 1 and portfolio 2

The first comparison will be as at 15-Jul-2023, so that we can see the effect of cashflow entitlement dates for the different portfolios. However, we will also compare the accrued interest as at 15-Jun-2023 to confirm that the entitlement to accrued interest for the month of June is consistent between the two portfolios.

To complete this demo, we will:
 - Sell on the 22-Jun-2023 4,000,000 USD (original notional value) the position held in portfolio 1
 - Confirm that the entitlement to the past (and still unpaid) coupon has remained unchanged but the entitlements to future cashflows and accrued interest have halved
 
In all transactions, we are paying / receiving 100,000 USD per 1,000,000 USD of original notional value - this generates the additional cash entry in the valuation requests.


In [ ]:
trade_date_1 = datetime.strptime("2023-05-28T00:00:00+00:00","%Y-%m-%dT%H:%M:%S%z")
trade_date_2 = datetime.strptime("2023-06-03T00:00:00+00:00","%Y-%m-%dT%H:%M:%S%z")
sell_date = datetime.strptime("2023-06-22T00:00:00+00:00","%Y-%m-%dT%H:%M:%S%z")

effective_at_date = datetime.strptime("2023-07-15T10:00:00+00:00","%Y-%m-%dT%H:%M:%S%z")
window_start_date = datetime.strptime("2023-06-01T10:00:00+00:00","%Y-%m-%dT%H:%M:%S%z")
window_end_date = datetime.strptime("2023-08-01T10:00:00+00:00","%Y-%m-%dT%H:%M:%S%z")


In [ ]:
open_position_1=[lm.TransactionRequest(
        transaction_id="txn_1",
        type="Buy",
        instrument_identifiers={
            "Instrument/default/ClientInternal" : "US3136AKAD59",
            "Instrument/default/Isin" : "US3136AKAD59"
        },
        transaction_date=trade_date_1.isoformat(),
        settlement_date=trade_date_1.isoformat(),
        units=8,
        transaction_price=lm.TransactionPrice(price=1, type="Price"),
        total_consideration=lm.CurrencyAndAmount(amount=800000, currency="USD"),
        transaction_currency="USD",
        properties={}
    )]

response = transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code="portfolio_1",
    transaction_request=open_position_1
)

print("Buy transaction upserted successfully for portfolio_1")

In [ ]:
open_position_2=[lm.TransactionRequest(
        transaction_id="txn_2",
        type="Buy",
        instrument_identifiers={
            "Instrument/default/ClientInternal" : "US3136AKAD59",
            "Instrument/default/Isin" : "US3136AKAD59"
        },
        transaction_date=trade_date_2.isoformat(),
        settlement_date=trade_date_2.isoformat(),
        units=8,
        transaction_price=lm.TransactionPrice(price=1, type="Price"),
        total_consideration=lm.CurrencyAndAmount(amount=800000, currency="USD"),
        transaction_currency="USD",
        properties={}
    )]

response = transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code="portfolio_2",
    transaction_request=open_position_2
)

print("Buy transaction upserted successfully for portfolio_2")

# 6. Cash Flows and Entitlements

Before we can check the cashflows of our first two transactions, we need to define a recipe that can pick up the pool factors and that specifies the valuation model as "Constant Time Value of Money".

In [ ]:
recipe_code = "TESTMBS"

pool_factor_data_rule = lu.models.MarketDataKeyRule(
                        key = 'Quote.Isin.*',
                        supplier = 'Lusid',
                        data_scope = scope,
                        quote_type = 'PoolFactor',
                        field = 'mid')

resp = recipes_api.upsert_configuration_recipe(
    upsert_recipe_request = lu.models.UpsertRecipeRequest(
        configuration_recipe = lu.models.ConfigurationRecipe(
            scope = scope,
            code = recipe_code,
            market = lu.models.MarketContext(
                market_rules = [
                    pool_factor_data_rule
                ],
                options = lu.models.MarketOptions(
                    default_scope = 'default',
                    calendar_scope = scope
                )
            ),
            pricing = lu.models.PricingContext(
                options = lu.models.PricingOptions(
                    model_selection = lu.models.ModelSelection(
                        library = 'Lusid',
                        model = 'ConstantTimeValueOfMoney'
                    ),
                    allow_partially_successful_evaluation = True
                )
            )
        )
    )
)

Now we build the queries for the portfolio cash flows:

In [ ]:
upsertable_cash_flows_portfolio_1 = transaction_portfolios_api.get_upsertable_portfolio_cash_flows(
    scope=scope,
    code="portfolio_1",
    effective_at=effective_at_date,
    window_start=window_start_date,
    window_end=window_end_date,
    recipe_id_scope=scope,
    recipe_id_code=recipe_code
)
lusid_response_portfolio_1=lusid_response_to_data_frame(upsertable_cash_flows_portfolio_1)

upsertable_cash_flows_portfolio_2 = transaction_portfolios_api.get_upsertable_portfolio_cash_flows(
    scope=scope,
    code="portfolio_2",
    effective_at=effective_at_date,
    window_start=window_start_date,
    window_end=window_end_date,
    recipe_id_scope=scope,
    recipe_id_code=recipe_code
)
lusid_response_portfolio_2=lusid_response_to_data_frame(upsertable_cash_flows_portfolio_2)


Here is the response for portfolio 1 (as at 15-Jul-2023):

In [ ]:
lusid_response_portfolio_1[["transaction_id", "settlement_date", "units", "transaction_currency"]]

Here is the response for portfolio 2 (as at 15-Jul-2023), not entitled to the June cashflows:

In [ ]:
lusid_response_portfolio_2[["transaction_id", "settlement_date", "units", "transaction_currency"]]

As the first portfolio was bought before the end of the month of May-2023 and the second portfolio was bought after the end of that month, the two portfolios will have different entitlements in the month of June (namely, the second portfolio will not be entitled to the principal and interest payments due on the 25-June-2023).

However, in any date in June after the purchase of the second portfolio, the two portfolios are entitled to the same accrued interest. Let's confirm that by comparing the accrued interest for the two portfolios as at 15-Jun-2023:

In [ ]:
def run_lusid_valuation(portfolio_code, date):
    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code=recipe_code),
        metrics=[
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("Valuation/CurrentNotional", "Value"),
            lm.AggregateSpec("Holding/default/Accrual", "Value")
        ],
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[lm.PortfolioEntityId(scope=scope, code=portfolio_code)],
        valuation_schedule=lm.ValuationSchedule(effective_at=date.isoformat()),
    )
    val_data = aggregation_api.get_valuation(valuation_request=valuation_request).data
    vals_df = pd.DataFrame(val_data)
    vals_df.rename(
        columns={
            "Instrument/default/Name": "Instrument Name",
            "Instrument/default/ClientInternal": "ISIN",
            "Holding/default/Units": "Units Held",
            "Valuation/CurrentNotional": "Current Notional per Unit",
            "Holding/default/Accrual": "Accrued Interest"
        },
        inplace=True,
    )
    try:
        return vals_df.drop("Aggregation/Errors", axis=1)
    except:
        return vals_df

In [ ]:
effective_at_date = datetime.strptime("2023-06-15T10:00:00+00:00","%Y-%m-%dT%H:%M:%S%z")
print("Portfolio 1 Accrued Interest Profile as at 15-Jun-2023")
run_lusid_valuation("portfolio_1", effective_at_date)

In [ ]:
print("Portfolio 2 Accrued Interest Profile as at 15-Jun-2023")
run_lusid_valuation("portfolio_2", effective_at_date)

To further demonstrate how the entitlements for MBS cashflows work in lusid, we will now sell half of the portfolio 1 and get the past and future cashflows and accrued interest, as at 15-Jul-2023. 

Notice now that the entitlement cashflows due on the 25-Jun-2023 have remained unchanged (as the entitlement boundary is the beginning of the month of June) but the values to be received on the 25-Jul-2023 have halved (as the halving transaction happened in June, before the entitlement boundary date for July cashflows).

In [ ]:
sell_half_position_1=[lm.TransactionRequest(
        transaction_id="txn_3",
        type="Sell",
        instrument_identifiers={
            "Instrument/default/ClientInternal" : "US3136AKAD59",
            "Instrument/default/Isin" : "US3136AKAD59"
        },
        transaction_date=sell_date.isoformat(),
        settlement_date=sell_date.isoformat(),
        units=4,
        transaction_price=lm.TransactionPrice(price=1, type="Price"),
        total_consideration=lm.CurrencyAndAmount(amount=400000, currency="USD"),
        transaction_currency="USD",
        properties={}
    )]

response = transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code="portfolio_1",
    transaction_request=sell_half_position_1
)

print("Sell transaction upserted successfully for portfolio_1")

In [ ]:
effective_at_date = datetime.strptime("2023-07-15T10:00:00+00:00","%Y-%m-%dT%H:%M:%S%z")

upsertable_cash_flows_portfolio_1_post_sale = transaction_portfolios_api.get_upsertable_portfolio_cash_flows(
    scope=scope,
    code="portfolio_1",
    effective_at=effective_at_date,
    window_start=window_start_date,
    window_end=window_end_date,
    recipe_id_scope=scope,
    recipe_id_code=recipe_code
)
lusid_response_portfolio_1_post_sale=lusid_response_to_data_frame(upsertable_cash_flows_portfolio_1_post_sale)

lusid_response_portfolio_1_post_sale[["transaction_id", "settlement_date", "units", "transaction_currency"]]

# 7. Valuations

In addition from querying future cashflows, we may want to value our portfolio on a daily basis using market quotes and obtain a daily P&L. To do that, we now move from the "Constant Time Value of Money" model into the "Simple Static" model.

The first step is to define appropriate recipes and quote upserting functions.

In [ ]:
quoted_price_data = [["US3136AKAD59", "2023-07-05", 100, "USD"],
                    ["US3136AKAD59", "2023-07-06", 100.1, "USD"],
                    ["US3136AKAD59", "2023-07-07", 99.93, "USD"]]
bond_quotes = pd.DataFrame(quoted_price_data, columns=["ISIN","Date","Price","Currency"])
latest_pool_factor = 0.095096

A important point to note is that Simple Static only accepts one quote (the price) for valuation. To represent correctly the latest MBS price, we need to include the latest known pool factor value as the scaling factor of the quote, as in the recipe below.

In [ ]:
# Create quotes request
instrument_quotes = {
    index: lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider="Lusid",
                instrument_id=row["ISIN"],
                instrument_id_type="ClientInternal",
                quote_type="Price",
                field="mid",
            ),
            effective_at=datetime.strptime(row["Date"],"%Y-%m-%d").isoformat()+'Z',
        ),
        metric_value=lm.MetricValue(value=row["Price"], unit=row["Currency"]),
        scale_factor=100/(original_notional*latest_pool_factor),
    )
    for index, row in bond_quotes.iterrows()
}

# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

if response.failed == {}:
    print(
        f"Quote successfully loaded into LUSID. {len(response.values)} quotes loaded."
    )

else:
    print(
        f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID."
    )

In [ ]:
# Create a recipe to perform a valuation
recipe_code_for_simple_static = "MBS_SIMPLE_STATIC"

quoted_price_key_rule = lm.MarketDataKeyRule(
                key="Credit.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D")


configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code=recipe_code_for_simple_static,
    market=lm.MarketContext(
        market_rules=[
            pool_factor_data_rule,
            quoted_price_key_rule
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="ComplexBond",
                parameters="{}",
            )
        ]
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

In [ ]:
def get_val(date, portfolio_code):

    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code=recipe_code_for_simple_static),
        metrics=[
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            lm.AggregateSpec("Quotes/Price", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("Valuation/PV/Amount", "Value"),
            lm.AggregateSpec("Holding/default/Accrual", "Value"),
            lm.AggregateSpec("Valuation/PnL/Tm1", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[lm.PortfolioEntityId(scope=scope, code=portfolio_code)],
        valuation_schedule=lm.ValuationSchedule(effective_at=date),
    )

    val_data = aggregation_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)

    vals_df.rename(
        columns={
            "Instrument/default/Name": "InstrumentName",
            "Instrument/default/ClientInternal": "ISIN",
            "Valuation/PV/Amount": "Present Value",
            "Valuation/PnL/Tm1": "PnL (1-day)",
            "Holding/default/Accrual": "Accrued Interest",
        },
        inplace=True,
    )
    try:
        return vals_df.drop("Aggregation/Errors", axis=1)
    except:
        return vals_df

And we complete this demo by showing what is the value of our MBS position on the second day (06-Jul-2023) and the PnL of our portfolio compared to the previous day (05-Jul-2023). The PnL includes accrued interest and market moves.

In [ ]:
get_val("2023-07-06T10:00:00Z", "portfolio_1")